# libaray

## Structure Learning or Defining

In [ ]:
from graphviz import Digraph
# Create Digraph object
dot = Digraph()

# Nodes
dot.node('Smoker')
dot.node('Gender')
dot.node('Age')
dot.node('Mutation')
dot.node('Family')
dot.node('Cancer')
dot.node('Treatment')
dot.node('Stage')
dot.node('Relapse')

# Edges
lung_cancer_edges = [('Smoker', 'Mutation'), 
('Age', 'Smoker'), ('Age', 'Mutation'), ('Age', 'Relapse'),
('Gender', 'Smoker'), ('Gender', 'Mutation'),
('Mutation', 'Treatment'),
('Treatment', 'Relapse'),
('Family', 'Mutation'), ('Family', 'Relapse'),
('Stage', 'Treatment'), ('Stage', 'Relapse')]

for head, tail in lung_cancer_edges:
    dot.edge(head, tail)

dot


# Experiments

In [2]:
a = '''0   0   1   1   0   1   0   0   0
   1   0   1   0   0   1   0   0   0
   0   0   0   0   0   0   0   0   0
   0   0   0   0   1   0   0   1   0
   0   0   1   0   0   0   0   0   0
   0   0   0   0   0   0   0   1   0
   1   0   1   0   0   1   0   0   1
   0   0   0   0   1   0   0   0   0
   0   1   0   1   1   1   0   0   0'''

from bn_library import mcmc_edges
mcmc_edges(a)

Index(['Age', 'Gender', 'Smoker', 'Stage', 'Family', 'Cancer', 'Mutation',
       'FamilyTypeNum', 'Relapse'],
      dtype='object')
['Age', 'Gender', 'Smoker', 'Stage', 'Family', 'Cancer', 'Mutation', 'FamilyTypeNum', 'Relapse']


[('A', 'S'),
 ('A', 'S1'),
 ('A', 'C'),
 ('G', 'A'),
 ('G', 'S'),
 ('G', 'C'),
 ('S1', 'F'),
 ('S1', 'FN'),
 ('F', 'S'),
 ('C', 'FN'),
 ('M', 'A'),
 ('M', 'S'),
 ('M', 'C'),
 ('M', 'R'),
 ('FN', 'F'),
 ('R', 'G'),
 ('R', 'S1'),
 ('R', 'F'),
 ('R', 'C')]

In [3]:
from sympy import im
import bn_library as bn
import pandas as pd
import numpy as np
# import mcmc_structure_learn as mcmc

structure_methods = {"ex": bn.exhaustive_search, 'hc': bn.hill_climate, 'tan': bn.TAN, 'k2': bn.K2, 'pc': bn.constraint_based, 'mmhc': bn.mmhc}
def structure_learn(data, Y):
    strus = {}
    for key, method in structure_methods.items():
        if 'ex' == key:
            continue
        if 'tan' == key:
            stru = structure_methods[key](data, Y, 'bic')
        elif 'pc' == key:
            stru = structure_methods[key](data)
        else:
            stru = structure_methods[key](data, 'bic')
        strus[key] = stru
    return strus
    
# Data load
# df = pd.read_csv("query4_binary_Y_Feature.csv")

# from pgmpy.estimators import HillClimbSearch
# from pgmpy.estimators import K2Score, BicScore
# hc = HillClimbSearch(df)
# best_model = hc.estimate()
# print(best_model.edges())


import bnlearn
df = bnlearn.import_example()[:500]



DAG = bn.structure_learning.fit(df, methodtype='hc', root_node='Survived', bw_list_method='nodes', verbose=3)

print(DAG)

# Plot
G = bn.plot(DAG)

# Parameter learning
model = bn.parameter_learning.fit(DAG, df, verbose=3);

# # Bayesian Network training
# Y = 'Mutation'
# struts = structure_learn(df, Y)
# models = {key: bn.MLE(stru, df) for key, stru in struts.items()}

# # evaluation
# results = {"model": [], "log_likelihood": [], "accuracy": [], 'auc': []}
# for key, model in models.items():
#     print(key)
#     results['model'].append(key)
#     results['log_likelihood'].append(bn.log_likelihood(model, df))
#     results['accuracy'].append(bn.accuracy(model, Y, df))
#     results['auc'].append(bn.auc(model, Y, df))
# pd.DataFrame(results)





[bnlearn] >Import dataset..


AttributeError: module 'bn_library' has no attribute 'structure_learning'

In [23]:
import bnlearn as bn
df = bn.import_example()[:500]
import pandas as pd

DAG = bn.structure_learning.fit(df, methodtype='hc', root_node='Survived', bw_list_method='nodes', verbose=3)

# print(DAG)
# print(type(DAG['model']))
# print(DAG['model'])
print(DAG['model_edges'])


def edge2adjmat(edges):
    nodes = list(set([item for t in edges for item in t]))
    mat = [[False] * len(nodes)] * len(nodes)
    for e in edges:
        mat[nodes.index(e[0])][nodes.index(e[1])] = True
    return pd.DataFrame(mat, index=nodes, columns=nodes)

from pgmpy.models import BayesianModel
skeleton = [('Cloudy', 'Rain'), ('Sprinkler', 'Wet_Grass'), ('Sprinkler', 'Cloudy'), ('Rain', 'Wet_Grass')]
DAG = {"model_edges": skeleton, 
'adjmat': edge2adjmat(skeleton),
'model': None}
# 'model': BayesianModel(skeleton)}
# print(edge2adjmat(skeleton))
# print(df.columns)

# Plot
# G = bn.plot(DAG)
# Parameter learning
model = bn.parameter_learning.fit(DAG, df, verbose=3);


[bnlearn] >Import dataset..
[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bic]
[('Cloudy', 'Rain'), ('Sprinkler', 'Wet_Grass'), ('Sprinkler', 'Cloudy'), ('Rain', 'Wet_Grass')]
[bnlearn] >Parameter learning> Computing parameters using [bayes]
[bnlearn] >Conversion of adjmat to BayesianNetwork.


NetworkXError: Input is not a valid edge list

## Parameter Learning

In [ ]:
def check_node_feature_names(edges, data):
    features = set(data.head().columns)
    node_names = set([node for node_pair in edges for node in node_pair])
    assert(not features.difference(node_names))

def paras_learn_by_bayes(edges, data):
    check_node_feature_names(edges, data)
    # Make the actual Bayesian DAG
    DAG = bn.make_DAG(edges, verbose=0, methodtype='bayes')
    model = bn.parameter_learning.fit(DAG, data, verbose=3, methodtype='bayes')
    bn.print_CPD(model)
    return model

def paras_learn_by_ML(edges, data):
    check_node_feature_names(edges, data)
    DAG = bn.make_DAG(edges, verbose=0, methodtype='bayes')
    # print(DAG)
    # print(set(data.head().columns))
    model = bn.parameter_learning.fit(DAG, data, methodtype='ml')
    bn.print_CPD(model)
    return model

edges = [('Cloudy', 'Sprinkler'),
         ('Cloudy', 'Rain'),
         ('Sprinkler', 'Wet_Grass'),
         ('Rain', 'Wet_Grass')]

# paras_learn_by_bayes(edges, df)
# paras_learn_by_ML(edges, df)

## BN learning

In [ ]:
def bn_learn_by_data(data, structure_setting, para_settting):
    if structure_setting == 'cs':
        model = bn.structure_learning.fit(data, methodtype='cs')
    else:
        model = bn.structure_learning.fit(data, methodtype=structure_setting, scoretype='bic')
    # prune insignificant edges
    model = bn.independence_test(model, data, alpha=0.05, prune=True)
    model_bn = bn.parameter_learning.fit(model, df, methodtype=para_settting)  # maximum likelihood estimator
    return model_bn

def bn_defined():
    from pgmpy.factors.discrete import TabularCPD
    edges = structure_defined()
    DAG = bn.make_DAG(edges)
    # Cloudy
    cpt_cloudy = TabularCPD(variable='Cloudy', variable_card=2, values=[[0.3], [0.7]])
    print(cpt_cloudy)

    # Sprinkler
    cpt_sprinkler = TabularCPD(variable='Sprinkler', variable_card=2,
                            values=[[0.4, 0.9], [0.6, 0.1]],
                            evidence=['Cloudy'], evidence_card=[2])
    print(cpt_sprinkler)

    # Rain
    cpt_rain = TabularCPD(variable='Rain', variable_card=2,
                        values=[[0.8, 0.2], [0.2, 0.8]],
                        evidence=['Cloudy'], evidence_card=[2])
    print(cpt_rain)

    # Wet Grass
    cpt_wet_grass = TabularCPD(variable='Wet_Grass', variable_card=2,
                            values=[[1, 0.1, 0.1, 0.01],
                                    [0, 0.9, 0.9, 0.99]],
                            evidence=['Sprinkler', 'Rain'],
                            evidence_card=[2, 2])
    DAG = bn.make_DAG(DAG, CPD=[cpt_cloudy, cpt_sprinkler, cpt_rain, cpt_wet_grass])
    return DAG

In [ ]:
# Save model
bn.save(model, filepath='bnlearn_model', overwrite=True)
# Learn structure
model = bn.structure_learning.fit(df, methodtype='tan', class_node='lung')# Load model
model = bn.load(filepath='bnlearn_model')

In [41]:
import pandas as pd

df = pd.DataFrame({"a": [1,23,4,5], "b":[2,34,54,6]})

df["a"].values.tolist()
print(df.corr())

for k, row in df.iterrows():
    print(row.to_dict())
# df.drop('a', axis=1, inplace=True)
print(df['a'].value_counts().keys().to_list())


          a         b
a  1.000000  0.323835
b  0.323835  1.000000
{'a': 1, 'b': 2}
{'a': 23, 'b': 34}
{'a': 4, 'b': 54}
{'a': 5, 'b': 6}
[1, 23, 4, 5]


In [24]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
student = BayesianNetwork([('diff', 'grade'), ('intel', 'grade')])
cpd = TabularCPD('grade', 2, [[0.1, 0.9, 0.2, 0.7],
                                      [0.9, 0.1, 0.8, 0.3]],
                          ['intel', 'diff'], [2, 2])
student.add_cpds(cpd)
student.get_cpds()

[<TabularCPD representing P(grade:2 | intel:2, diff:2) at 0x7fc7516fad60>]

In [ ]:
import numpy as np
from sklearn import metrics

y = np.array(['a', 'b', 'a', 'b'])
y_dict = {e: i for i, e in enumerate(set(y))}
scores = np.array(['a', 'a', 'a', 'b'])

y = [y_dict[e] for e in y]
scores = [y_dict[e] for e in scores]

fpr, tpr, thresholds = metrics.roc_curve(y, scores)
metrics.auc(fpr, tpr)

In [12]:
# Load example DAG with CPD
model = bn.import_DAG('sprinkler', CPD=True)

# Take 1000 samples from the CPD distribution
df = bn.sampling(model, n=500)

df.to_csv('sample.csv', index=False)

[bnlearn] >Import <sprinkler>
[bnlearn] >Checking CPDs..
[bnlearn] >Check for DAG structure. Correct: True
[bnlearn] >Forward sampling for 500 samples..


  0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
import bnlearn as bn
model = bn.import_DAG('sprinkler', CPD=True)

# Take 1000 samples from the CPD distribution
df = bn.sampling(model, n=1000)
df.loc[len(df)] = [2, 1, 0, 1]
df.loc[len(df)] = [3, 1, 0, 1]
df.loc[len(df)] = [4, 3, 2, 3]


print(df.columns)
skeleton = [('Cloudy', 'Rain'), ('Sprinkler', 'Wet_Grass'), ('Rain', 'Wet_Grass')]
from pgmpy.estimators import BayesianEstimator
from pgmpy.metrics import log_likelihood_score
model = BayesianModel(skeleton)
model.fit(df, estimator=BayesianEstimator, prior_type="BDeu")
log_likelihood_score(model, df)




[bnlearn] >Import <sprinkler>
[bnlearn] >Checking CPDs..
[bnlearn] >Check for DAG structure. Correct: True
[bnlearn] >Forward sampling for 1000 samples..


  0%|          | 0/4 [00:00<?, ?it/s]

1    549
0    451
2      1
3      1
4      1
Name: Cloudy, dtype: int64
Index(['Cloudy', 'Sprinkler', 'Rain', 'Wet_Grass'], dtype='object')


-2005.8161942341667

In [40]:
import pyAgrum as gum
import pandas as pd
import random
# reference https://agrum.gitlab.io/articles/agrumpyagrum-0229-and-dataframe.html

train_data = pd.read_csv('sample.csv')

s_learner = gum.BNLearner(train_data)  # creates a learner by passing the dataframe
# s_learner.useGreedyHillClimbing()     # sets a local-search algorithm for the structural learning
s_learner.useK2(random.shuffle([i for i in range(len(train_data.columns))]))
s_learner.useScoreBIC()               # sets BIC score as the metric
structure = s_learner.learnBN()       # learning the structure

id2name = {structure.idFromName(node_n): node_n for node_n in structure.names()}
[(id2name[ele[0]], id2name[ele[1]]) for ele in structure.arcs()]

# print(structure.arcs())
# print(structure.nodes())
# # print(train_data.columns)
# print(structure.names())
# # print(structure.cpt('Rain'))
# print(structure.idFromName('Rain'))
# print(structure.toDot())
# structure.dag()

# bn_learned = gum.BayesNet(structure)                   # initializing the bn with the learned structure
# p_learner = gum.BNLearner(train_data, structure)  # crates a learner to learn parameters
# p_learner.useEM(1e-10)                                 # sets EM to learn parameters
# p_learner.fitParameters(bn_learned)                            # learning the parameters

[('Rain', 'Wet_Grass'),
 ('Cloudy', 'Rain'),
 ('Sprinkler', 'Wet_Grass'),
 ('Cloudy', 'Sprinkler')]

In [60]:
s1 = '''0   1   0   1   0   1   0   0   1
0   0   1   1   1   1   0   0   0
0   0   0   0   1   0   0   1   0
0   0   1   0   0   0   0   0   0
0   0   0   0   0   1   0   1   0
0   0   0   0   0   0   0   1   0
1   1   1   0   1   1   0   0   1
0   0   0   0   0   0   0   0   0
0   1   1   1   1   1   0   0   0'''

s2 = '''0   1   1   1   0   1   0   0   1
   0   0   1   0   1   1   0   0   0
   0   0   0   1   0   0   0   1   0
   0   0   0   0   0   0   0   0   0
   0   0   1   0   0   1   0   1   0
   0   0   0   0   0   0   0   1   0
   1   0   1   1   1   1   0   0   1
   0   0   0   0   0   0   0   0   0
   0   1   1   1   1   1   0   0   0'''
def mcmc_edges(a):
    '''a is a str representing the adjacent matrix'''
    brief = {'M':'Mutation', 'FN':'FamilyTypeNum', 'R':'Relapse', 'A': 'Age', 'C':'Cancer', 
'F':'Family', 'S1' :'Stage', 'G': 'Gender', 'S':'Smoker'}
    long_s = {v:k for k, v in brief.items()}
    b = a.split()
    N = 9
    import numpy as np
    import pandas as pd
    b = np.array([int(b[i*N+j]) for i in range(N) for j in range(N)]).reshape((N,N))

    df = pd.read_csv('query4R.csv')
    print(df.columns)

    cols = df.columns.to_list()
    print(cols)

    edges = []
    for i in range(N):
        for j in range(N):
            if b[i][j] == 1:
                edges.append((cols[i], cols[j]))
    edges = [(long_s[ele[0]],long_s[ele[1]]) for ele in edges]
    return edges
mcmc_edges(s2)



Index(['Age', 'Gender', 'Smoker', 'Stage', 'Family', 'Cancer', 'Mutation',
       'FamilyTypeNum', 'Relapse'],
      dtype='object')
['Age', 'Gender', 'Smoker', 'Stage', 'Family', 'Cancer', 'Mutation', 'FamilyTypeNum', 'Relapse']


[('A', 'G'),
 ('A', 'S'),
 ('A', 'S1'),
 ('A', 'C'),
 ('A', 'R'),
 ('G', 'S'),
 ('G', 'F'),
 ('G', 'C'),
 ('S', 'S1'),
 ('S', 'FN'),
 ('F', 'S'),
 ('F', 'C'),
 ('F', 'FN'),
 ('C', 'FN'),
 ('M', 'A'),
 ('M', 'S'),
 ('M', 'S1'),
 ('M', 'F'),
 ('M', 'C'),
 ('M', 'R'),
 ('R', 'G'),
 ('R', 'S'),
 ('R', 'S1'),
 ('R', 'F'),
 ('R', 'C')]